In [2]:
#순환 신경망의 간단한 예시
sentence = "나는 밥을 먹었다."

source_sentence = "<Start>" + sentence
target_sentence = sentence + "end"

print("Source 문장 : ", source_sentence)
print("Target 문장 : ", target_sentence)

Source 문장 :  <Start>나는 밥을 먹었다.
Target 문장 :  나는 밥을 먹었다.end


In [3]:
#데이터 실습
import os, re
import numpy as np
import tensorflow as tf

file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f: #파일을 읽기모드로 open
    raw_corpus = f.read().splitlines() #줄 단위로 끊어서 list형태로 읽어옴
    
print(raw_corpus[:9]) #출력

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [4]:
#원치않는 문장(화자, 공백)을 제거
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue #문장의 길이가 0이면 pass
    if sentence[-1] == ":": continue # 문장의 끝이 ":"이면 pass  
    if idx > 9: break #idx 가 n개가 되면 중단
    
    print(sentence)

#1차 오류
# Resolved. resolved 가 출력됨
# 복사문과의 차이점을 찾자
#print문이 for문 밖에 있어서 제대로 출력되지 않았다.

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [5]:
#데이터 정제
def preprocess_sentence(sentence): #문장에서 단어를 추출하기 위한 전처리 과정
    sentence = sentence.lower().strip() #모든 문자를 소문자화 하고 공백을 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #특수문자 양쪽에 공백을 삽입
    sentence = re.sub(r'[" "]+', " ", sentence) #여러개의 공백은 하나의 공백으로 변경
    sentence = re.sub(r'[^a-zA-Z?.!,¿]+', " ", sentence) #대소문자, 특수문자가 아닌 문자는 하나의 공백으로 변경
    sentence = sentence.strip() #공백을 다시 제거
    sentence = '<start> ' + sentence + ' <end>' #문장의 양끝에 시작점과 끝점 표시
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence.")) #예시1

<start> this is sample sentence . <end>


In [6]:
#데이터 구축
corpus = [] #정제된 문장을 모으는 리스트

for sentence in raw_corpus:
    #위에서 적용했던 공백, 화자 제거
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    #문장 전처리
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    

#출력하여 결과 확인
for i in range(10):
    print(corpus[i])

<start> before we proceed any further , hear me speak . <end>
<start> speak , speak . <end>
<start> you are all resolved rather to die than to famish ? <end>
<start> resolved . resolved . <end>
<start> first , you know caius marcius is chief enemy to the people . <end>
<start> we know t , we know t . <end>
<start> let us kill him , and we ll have corn at our own price . <end>
<start> is t a verdict ? <end>
<start> no more talking on t let it be done away , away ! <end>
<start> one word , good citizens . <end>


In [7]:
#단어를 토큰화하여 사전으로 만들고 데이터를 숫자로 변화하는 벡터화 진행

#데이터 토큰화 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words = 7000, #7000개의 단어를 토큰화
    filters = ' ', #위에서 문장을 정제 했기 때문에 필터가 필요없음
    oov_token = "<unk>"
    )

    tokenizer.fit_on_texts(corpus) #내부의 단어장을 완성
    tensor = tokenizer.texts_to_sequences(corpus) #corpus를 tensor로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa79c4cfa90>


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
#2행이 start이라서 모든 텐서의 시작이 2로 시작되는 사실을 알 수 있다!

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [9]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:] #tensor에서 start를 제거

print(src_input[0])
print(tgt_input[0])
#입력 시퀀스 길이보다 문장이 짧으면 '0'<pad>가 된다.

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [10]:
#모델 생성
BUFFER_SIZE = len(src_input) #입력값의 길이 크기의 버퍼
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 #생성한 사전의 단어 7000개와 <pad>포한한 7000+1개의 크기

#데이터 소스로부터 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) 
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [11]:
#모델의 구조를 클래스화 한 것?
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 2.76763720e-04,  1.89827872e-04, -1.12452894e-04, ...,
          1.22651982e-05,  9.38863013e-05,  2.42295748e-04],
        [ 5.24333969e-04,  6.39118953e-04, -3.65780550e-04, ...,
          3.24963912e-04,  3.42263957e-04,  2.86425347e-04],
        [ 5.68508753e-04,  9.80879180e-04, -6.25625602e-04, ...,
          3.33377451e-04,  3.95896466e-04,  1.96551293e-04],
        ...,
        [ 5.42455760e-04,  1.24653394e-03,  2.75897910e-03, ...,
          2.17727458e-04, -3.21157416e-03,  1.37832935e-03],
        [ 6.60510210e-04,  1.39664009e-03,  3.13341361e-03, ...,
          2.60383560e-04, -3.29506840e-03,  1.65270700e-03],
        [ 7.62404583e-04,  1.54103537e-03,  3.48428357e-03, ...,
          3.11897573e-04, -3.34579009e-03,  1.91131548e-03]],

       [[ 2.76763720e-04,  1.89827872e-04, -1.12452894e-04, ...,
          1.22651982e-05,  9.38863013e-05,  2.42295748e-04],
        [ 3.76568350e-04,  3.60022270e-04, -2.

In [12]:
#모델 정보 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
#모델 학습 시작 (오래 걸리니 한번만 하자)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 18s 172ms/step - loss: 3.5352
Epoch 2/30
93/93 [==============================] - 17s 178ms/step - loss: 2.8201
Epoch 3/30
93/93 [==============================] - 17s 183ms/step - loss: 2.7366
Epoch 4/30
93/93 [==============================] - 18s 190ms/step - loss: 2.6450
Epoch 5/30
93/93 [==============================] - 18s 198ms/step - loss: 2.5742
Epoch 6/30
93/93 [==============================] - 18s 197ms/step - loss: 2.5230
Epoch 7/30
93/93 [==============================] - 18s 192ms/step - loss: 2.4798
Epoch 8/30
93/93 [==============================] - 18s 192ms/step - loss: 2.4333
Epoch 9/30
93/93 [==============================] - 18s 194ms/step - loss: 2.3910
Epoch 10/30
93/93 [==============================] - 18s 196ms/step - loss: 2.3501
Epoch 11/30
93/93 [==============================] - 18s 195ms/step - loss: 2.3082
Epoch 12/30
93/93 [==============================] - 18s 194ms/step - loss: 2.2698
Epoch 13/30
9

In [14]:
#모델 평가
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

generate_text(model, tokenizer, init_sentence="<start> he") #실제로 문장을 생성

'<start> he is a <unk> , and i am a thing , <end> '

In [15]:
#데이터 읽어오기
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #파일의 경로
txt_list = glob.glob(txt_file_path) #폴더안의 파일들을 리스트화

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #각 줄을 읽음
        raw_corpus.extend(raw) #읽은 줄을 리스트에 삽입

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

#데이터 정제
corpus = [] #정제된 문장을 모으는 리스트

for sentence in raw_corpus:
    #위에서 적용했던 공백, 화자 제거
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    #문장 전처리
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#데이터 토큰화 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words = 13000, #13000개의 단어를 토큰화
    filters = ' ', #위에서 문장을 정제 했기 때문에 필터가 필요없음
    oov_token = "<unk>"
    )

    tokenizer.fit_on_texts(corpus) #내부의 단어장을 완성
    tensor = tokenizer.texts_to_sequences(corpus) #corpus를 tensor로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:] #tensor에서 start를 제거

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [   2   23   89 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa7008017c0>


In [16]:
#데이터 분류
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True,
                                                          random_state=34)
#분류된 데이터 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 346)
Target Train: (140599, 346)


In [26]:
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

#모델 학습 시작 (오래 걸리니 한번만 하자)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
93/93 [==============================] - 69s 715ms/step - loss: 3.3818
Epoch 2/10
93/93 [==============================] - 66s 710ms/step - loss: 2.7134
Epoch 3/10
93/93 [==============================] - 66s 710ms/step - loss: 2.5636
Epoch 4/10
93/93 [==============================] - 66s 713ms/step - loss: 2.4544
Epoch 5/10
93/93 [==============================] - 66s 712ms/step - loss: 2.3639
Epoch 6/10
93/93 [==============================] - 66s 713ms/step - loss: 2.2804
Epoch 7/10
93/93 [==============================] - 66s 712ms/step - loss: 2.1921
Epoch 8/10
93/93 [==============================] - 66s 712ms/step - loss: 2.0997
Epoch 9/10
93/93 [==============================] - 66s 711ms/step - loss: 1.9968
Epoch 10/10
93/93 [==============================] - 66s 711ms/step - loss: 1.8840


In [27]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
#<unk>는 왜 뜨는 걸까?
#일단 Unknow의 약자인듯 
#정확히는 영어가 아닌 다른 문자가 나오면 나오는 것 같음


#결과적으로 문법에는 맞지 않지만 문장이 완성됨 loss 1.8840

'<start> i love , and let of is wears i <end> '